In [ ]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from dataset import uniformDataset, contextUniformDataset
from model import MLP
from train import train_model, evaluate_model
from test import test_model

cuda = torch.cuda.is_available()
num_workers = 4 if cuda else 0 
device = torch.device("cuda" if cuda else "cpu")
verbose = True

In [ ]:
# TODO: real train/dev/test dataset
# TODO: training curves

In [ ]:
print("torch version: %s" % torch.__version__)
print("np version: %s" % np.__version__)
print("__debug__: %s" % __debug__)
print("cuda: %s" % cuda)
print("num_workers: %s" % num_workers)
print("device: %s" % device)
print("verbose: %s" % verbose)

In [ ]:
data_path = "../data"
path_trainx = "%s/train.npy" % data_path
path_trainy = "%s/train_labels.npy" % data_path
path_evalx = "%s/dev.npy" % data_path
path_evaly = "%s/dev_labels.npy" % data_path
path_testx = "%s/test.npy" % data_path

In [ ]:
evalx = np.load(path_evalx, allow_pickle=True)
evaly = np.load(path_evaly, allow_pickle=True)

# trainx = np.load(path_trainx, allow_pickle=True) # Note: for real training
# trainy = np.load(path_trainy, allow_pickle=True) # Note: for real training
trainx = evalx # Note: for development
trainy = evaly # Note: for development

testx = np.load(path_testx, allow_pickle=True)

In [ ]:
# hyper parameters
orig_x_dim, orig_y_dim = evalx[0].shape[1], 138
context_size = 5
input_size, output_size = orig_x_dim * (2*context_size+1), orig_y_dim
size_list = [input_size, 256, output_size] # TODO: change this
activation = nn.ReLU # use only one type of activation
lr = 1e-3 # default lr is 1e-3
epochs = 2

In [ ]:
# data loader
print("loading data...")

train_dataset = contextUniformDataset(trainx, trainy, context_size=context_size)
# eval_dataset = contextUniformDataset(evalx, evaly, context_size=context_size) # for real training
eval_dataset = train_dataset # for development
test_dataset = contextUniformDataset(testx, is_test=True, context_size=context_size)

# TODO: average #frames in one utterance in train dataset
batch_size = len(train_dataset) // train_dataset.utter_num

if verbose:
    print("train size: X:({}, {}) Y:({}, 1)".format(len(train_dataset), len(train_dataset[0][0]), len(train_dataset)))
    print("eval size: X:({}, {}) Y:({}, 1)".format(len(eval_dataset), len(eval_dataset[0][0]), len(eval_dataset)))
    print("test size: X:({}, {})".format(len(test_dataset), len(test_dataset[0])))

train_loader = DataLoader(
    train_dataset, # The dataset
    batch_size=batch_size,      # Batch size
    shuffle=True,      # Shuffles the dataset at every epoch
    pin_memory=True,   # Copy data to CUDA pinned memory
    num_workers=num_workers      # Number of worker processes for loading data.
                       )

eval_loader = DataLoader(
    eval_dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    num_workers=num_workers
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    num_workers=num_workers
)

In [ ]:
# model
print("building model...")

model = MLP(size_list, activation)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
# training
print("training...")
train_losses, eval_losses, eval_accs = \
    train_model(model, epochs, train_loader, eval_loader, criterion, optimizer, device)

In [ ]:
print("predicting...")
predicts = test_model(model, test_loader, device, save=True)
print("finished")